# Clean Issues Query Dataset

The dataset "042925_issues_query.csv" is provided by clearinghouse through their internal SQL query. This dataset contains the case issue and issue category for each case. This notebook outlines the steps I undertook to clean this dataset for building the ML model to predict case issues/issue categories.

# Import Libraries

In [ ]:
import json
import pandas as pd

# Load the dataset

In [2]:
issues = pd.read_csv("data/042925_issues_query.csv")
issues.head()

,doc_id,case_id,case_issue,case_issue_info
0,151963,45796,NaN,NaN
1,151946,43757,Candidate qualifications,"{""category"": ""Voting"", ""case_type"": ""VR"", ""des..."
2,151968,43296,NaN,NaN
3,151053,45726,Disability (inc. reasonable accommodations),"{""category"": ""Discrimination Basis"", ""case_typ..."
4,115014,17099,Release Requested,"{""category"": ""COVID-19"", ""case_type"": ""CJ|IM|J..."


# Clean the dataset

## The case_issue_info is in json format, parse it so we can separate out the issue category, case type, and description

In [3]:
def parse_json_column(val):
    if pd.isna(val) or val == '':
        return {}
    try:
        return json.loads(val)
    except (ValueError, SyntaxError):
        return {}

parsed = issues['case_issue_info'].apply(parse_json_column)
expanded = pd.json_normalize(parsed)
issues = issues.join(expanded[['category', 'case_type', 'description']])

issues.head()

,doc_id,case_id,case_issue,case_issue_info,category,case_type,description
0,151963,45796,NaN,NaN,NaN,NaN,NaN
1,151946,43757,Candidate qualifications,"{""category"": ""Voting"", ""case_type"": ""VR"", ""des...",Voting,VR,None
2,151968,43296,NaN,NaN,NaN,NaN,NaN
3,151053,45726,Disability (inc. reasonable accommodations),"{""category"": ""Discrimination Basis"", ""case_typ...",Discrimination Basis,CJ|CW|DR|EE|EJ|FA|FH|ID|IM|JC|JI|MH|NH|NS|PB|P...,None
4,115014,17099,Release Requested,"{""category"": ""COVID-19"", ""case_type"": ""CJ|IM|J...",COVID-19,CJ|IM|JC|JI|PN|PC|FA,


In [4]:
assert issues["case_issue_info"].isna().sum() == issues["category"].isna().sum()

## We do not care about case type and description, discard these along with the case_issue_info column as we already parsed it

In [5]:
issues = issues[["case_id", "doc_id", "case_issue", "category"]]

## Get a better understanding of the relationship between doc_id and case_issue

In [6]:
issues[issues["case_id"] == 45726]

,case_id,doc_id,case_issue,category
3,45726,151053,Disability (inc. reasonable accommodations),Discrimination Basis
50287,45726,150793,Disability (inc. reasonable accommodations),Discrimination Basis


In [7]:
issues[issues["case_id"] == 17099]

,case_id,doc_id,case_issue,category
4,17099,115014,Release Requested,COVID-19
5,17099,115014,Release Denied,COVID-19
6,17099,115014,Mitigation Denied,COVID-19
7,17099,115014,Undocumented immigrants - rights and duties,Immigration/Border
8,17099,115014,Employment,Immigration/Border
...,...,...,...,...
573033,17099,105699,Language access/needs,General/Misc.
573034,17099,105699,Forced labor,General/Misc.
573035,17099,105699,Disciplinary segregation,"Jails, Prisons, Detention Centers, and Other I..."
573036,17099,105699,Conditions of confinement,General/Misc.


I sampled some docs from the sampled cases to review them and don't believe there is any relationship between the doc_id and the case_issue. The case_issue is attached to the case level, not specifically associated with a single document. The apparent one-to-one mapping between doc_id and case_issue is an artificat from the join. 

## Remove the doc_id since we want to focus on the issues at case level.

In [8]:
issues = issues[["case_id", "case_issue", "category"]]

## Rename the category column to issue_category to avoid confusion

In [9]:
issues.rename(columns={'category': 'issue_category'}, inplace=True)

## Drop any rows that don't contain case_issue or issue_category & remove duplicates

In [10]:
assert issues["case_issue"].isna().sum() == issues["issue_category"].isna().sum()
len(issues)

786598

In [11]:
issues = issues[~issues["case_issue"].isna()].drop_duplicates()
len(issues)

67545

In [12]:
issues.head()

,case_id,case_issue,issue_category
1,43757,Candidate qualifications,Voting
3,45726,Disability (inc. reasonable accommodations),Discrimination Basis
4,17099,Release Requested,COVID-19
5,17099,Release Denied,COVID-19
6,17099,Mitigation Denied,COVID-19


## Do some preliminary EDA

- ~12K unique cases
- 405 unique issues, belonging to 22 issue categories
- Some cases have as many as 42 issues, but most cases have fewer than 10 issues
- Some cases have as many as 9 issue categories, but most cases have fewer than 5 issue categories

In [13]:
issues["case_id"].nunique()

11884

In [14]:
issues["case_issue"].nunique()

405

In [15]:
issues["issue_category"].nunique()

22

In [16]:
issues[["case_id", "case_issue"]].groupby("case_id").nunique().describe()

,case_issue
count,11884.000000
mean,5.681505
std,4.723344
min,1.000000
25%,2.000000
50%,5.000000
75%,7.000000
max,42.000000


In [17]:
issues[["case_id", "issue_category"]].groupby("case_id").nunique().describe()

,issue_category
count,11884.000000
mean,2.846096
std,1.506738
min,1.000000
25%,2.000000
50%,3.000000
75%,4.000000
max,9.000000


## Most issues belong to only one category, with the exception of 4 issues, obtain an understanding of why

In [18]:
issue_category_counts = issues.groupby('case_issue')['issue_category'].nunique()
issue_category_counts[issue_category_counts > 1]

case_issue
Funding    2
Other      3
Spanish    2
Testing    2
Name: issue_category, dtype: int64

## Issue == Funding

In [19]:
issues[issues["case_issue"] == "Funding"]["issue_category"].value_counts()

issue_category
General/Misc.          325
Reproductive rights      2
Name: count, dtype: int64

Ideally, within one case, we should have a one-to-one match between issue and issue category, find out any cases that have multiple matches

In [20]:
multi_match = issues[issues["case_issue"] == "Funding"][["case_id","issue_category"]].groupby("case_id").nunique()
multi_match[multi_match["issue_category"] > 1]

,issue_category
case_id,
45997,2


In [21]:
issues[(issues["case_issue"] == "Funding") & (issues["case_id"] == 45997)]

,case_id,case_issue,issue_category
10548,45997,Funding,Reproductive rights
10554,45997,Funding,General/Misc.


In [22]:
issues[issues["case_id"] == 45997]["case_issue"].value_counts()

case_issue
Funding                        2
Criminalization                1
Complete abortion ban          1
Abortion                       1
Medical care, general          1
EMTALA (42 U.S.C. § 1395dd)    1
Name: count, dtype: int64

In [23]:
issues[issues["case_id"] == 45997]["issue_category"].value_counts()

issue_category
Reproductive rights           4
Medical/Mental Health Care    1
Benefits (Source)             1
General/Misc.                 1
Name: count, dtype: int64

Case id 45997 appears to be mostly related to "Reproductive rights", for consistency, we'll drop "General/Misc." issue category.

In [24]:
issues.drop(10554, inplace=True)

In [25]:
multi_match = issues[issues["case_issue"] == "Funding"][["case_id","issue_category"]].groupby("case_id").nunique()
assert len(multi_match[multi_match["issue_category"] > 1]) == 0

## Issue == Spanish

In [26]:
issues[issues["case_issue"] == "Spanish"]["issue_category"].value_counts()

issue_category
Affected Language(s)                     57
Affected National Origin/Ethnicity(s)     5
Name: count, dtype: int64

Ideally, within one case, we should have a one-to-one match between issue and issue category, find out any cases that have multiple matches

In [27]:
multi_match = issues[issues["case_issue"] == "Spanish"][["case_id","issue_category"]].groupby("case_id").nunique()
multi_match[multi_match["issue_category"] > 1]

,issue_category
case_id,


## Issue == Testing

In [28]:
issues[issues["case_issue"] == "Testing"]["issue_category"].value_counts()

issue_category
Discrimination Area    88
General/Misc.          45
Name: count, dtype: int64

Ideally, within one case, we should have a one-to-one match between issue and issue category, find out any cases that have multiple matches

In [29]:
multi_match = issues[issues["case_issue"] == "Testing"][["case_id","issue_category"]].groupby("case_id").nunique()
multi_match[multi_match["issue_category"] > 1]

,issue_category
case_id,
829,2
10675,2
12446,2
13269,2
13407,2
13524,2
13569,2
13571,2
13573,2


In [30]:
for case_id in multi_match[multi_match["issue_category"] > 1].index.to_list():
    display(issues[(issues["case_issue"] == "Testing") & (issues["case_id"] == case_id)])

,case_id,case_issue,issue_category
397859,829,Testing,Discrimination Area
397861,829,Testing,General/Misc.


,case_id,case_issue,issue_category
65320,10675,Testing,Discrimination Area
65327,10675,Testing,General/Misc.


,case_id,case_issue,issue_category
75268,12446,Testing,Discrimination Area
75275,12446,Testing,General/Misc.


,case_id,case_issue,issue_category
242925,13269,Testing,Discrimination Area
242931,13269,Testing,General/Misc.


,case_id,case_issue,issue_category
122807,13407,Testing,General/Misc.
122820,13407,Testing,Discrimination Area


...

,case_id,case_issue,issue_category
188684,16414,Testing,Discrimination Area
188689,16414,Testing,General/Misc.


,case_id,case_issue,issue_category
88640,17579,Testing,Discrimination Area
88644,17579,Testing,General/Misc.


,case_id,case_issue,issue_category
230150,43993,Testing,Discrimination Area
230152,43993,Testing,General/Misc.


,case_id,case_issue,issue_category
88283,44163,Testing,Discrimination Area
88286,44163,Testing,General/Misc.


,case_id,case_issue,issue_category
57175,44308,Testing,Discrimination Area
57180,44308,Testing,General/Misc.


In [31]:
issues[issues["case_id"] == 829]["case_issue"].value_counts()

case_issue
Testing                   2
Medical care, general     1
Medical Exam / Inquiry    1
Name: count, dtype: int64

In [32]:
issues[issues["case_id"] == 829]["issue_category"].value_counts()

issue_category
Discrimination Area           2
Medical/Mental Health Care    1
General/Misc.                 1
Name: count, dtype: int64

All the cases appear to be assigned to both "Discrimination Area" and "General/Misc.
", for consistency, we'll drop "General/Misc." issue category.

In [33]:
issues = issues[~((issues['case_issue'] == 'Testing') & (issues['issue_category'] == 'General/Misc.'))]

In [34]:
multi_match = issues[issues["case_issue"] == "Testing"][["case_id","issue_category"]].groupby("case_id").nunique()
assert len(multi_match[multi_match["issue_category"] > 1]) == 0

## Issue == Other

In [35]:
issues[issues["case_issue"] == "Other"]["issue_category"].value_counts()

issue_category
Affected National Origin/Ethnicity(s)    77
General/Misc.                            46
Affected Language(s)                     20
Name: count, dtype: int64

Ideally, within one case, we should have a one-to-one match between issue and issue category, find out any cases that have multiple matches

In [36]:
multi_match = issues[issues["case_issue"] == "Other"][["case_id","issue_category"]].groupby("case_id").nunique()
multi_match[multi_match["issue_category"] > 1]

,issue_category
case_id,
2975,2
6616,2
15465,2
44183,2
45091,2


In [37]:
for case_id in multi_match[multi_match["issue_category"] > 1].index.to_list():
    display(issues[(issues["case_issue"] == "Other") & (issues["case_id"] == case_id)])

,case_id,case_issue,issue_category
179158,2975,Other,Affected Language(s)
179159,2975,Other,Affected National Origin/Ethnicity(s)


,case_id,case_issue,issue_category
352840,6616,Other,Affected Language(s)
352841,6616,Other,Affected National Origin/Ethnicity(s)


,case_id,case_issue,issue_category
404132,15465,Other,Affected Language(s)
404133,15465,Other,Affected National Origin/Ethnicity(s)


,case_id,case_issue,issue_category
78142,44183,Other,Affected National Origin/Ethnicity(s)
78144,44183,Other,Affected Language(s)


,case_id,case_issue,issue_category
17985,45091,Other,Affected National Origin/Ethnicity(s)
17986,45091,Other,Affected Language(s)


It's reasonable that multiple issue categories could have "other" issue, so leave these as is.

## Do another round of EDA

In [38]:
issue_category_counts = issues.groupby('case_issue')['issue_category'].nunique()
issue_category_counts[issue_category_counts > 1]

case_issue
Funding    2
Other      3
Spanish    2
Name: issue_category, dtype: int64

In [39]:
issues.groupby('issue_category')['case_issue'].nunique()

issue_category
Affected Language(s)                                          5
Affected National Origin/Ethnicity(s)                         6
Affected Race(s)                                              5
Affected Religion(s)                                          7
Affected Sex/Gender(s)                                        4
Benefits (Source)                                            15
COVID-19                                                     32
Death Penalty                                                 4
Disability and Disability Rights                             29
Discrimination Area                                          22
Discrimination Basis                                         13
EEOC-centric                                                  5
Environmental Justice and Resources                           7
General/Misc.                                                97
Immigration/Border                                           40
Jails, Prisons, Detention

## For each case_id and each issue_category, produce the list of case_issue

In [40]:
grouped = issues.groupby(["case_id", "issue_category"])["case_issue"].agg(list).reset_index()
grouped.head()

,case_id,issue_category,case_issue
0,2,General/Misc.,[Education]
1,3,Disability and Disability Rights,"[Intellectual/developmental disability, unspec..."
2,3,General/Misc.,"[Government services, Juveniles, Family reunif..."
3,3,"Jails, Prisons, Detention Centers, and Other I...","[Assault/abuse by staff (facilities), Placemen..."
4,5,Affected Sex/Gender(s),"[Female, Male]"


## Confirm no duplicate issue_category for each case_id

In [41]:
case_category_count = grouped[["case_id", "issue_category"]].groupby("case_id").value_counts()
assert len(case_category_count[case_category_count > 1]) == 0

## To make it easier to map case to issue category and issues, put the issue data into a dict

In [42]:
issue_data = grouped.groupby('case_id').apply(
    lambda x: dict(zip(x['issue_category'], x['case_issue']))
).reset_index(name='issue_data')

issue_data.head()

/var/folders/rx/3t2jtk8j69j_db_cydq_v6wc0000gn/T/ipykernel_48587/4166144190.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  issue_data = grouped.groupby('case_id').apply(


,case_id,issue_data
0,2,{'General/Misc.': ['Education']}
1,3,{'Disability and Disability Rights': ['Intelle...
2,5,"{'Affected Sex/Gender(s)': ['Female', 'Male'],..."
3,6,{'Disability and Disability Rights': ['Learnin...
4,7,"{'General/Misc.': ['Failure to train', 'Family..."


In [43]:
assert len(issue_data) == issue_data["case_id"].nunique()

In [44]:
len(issue_data)

11884

## Save the issue data for future use

In [45]:
issue_data.to_json("data/case_issues.json")